<a href="https://colab.research.google.com/github/maagdaidziak/ADN/blob/main/131931_magdalena_idziak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Wstęp

W projekcie podjęłam się predykcji cen mieszkań na wynajem w Poznaniu (nick na Kaggle Magdalena Idziak, wynik 112342.75),

Brakujące dane zaimputowałam na postawie tekstu z innych kolumna (np. liczbę pokoi, dzielnicę z tytułu ogłoszenia), metodą knn, oraz średniej. Dla danych typu bool brakujące kolumny uzupełniłam jako FALSE.

Następnie do predykcji najlepszą metodą okazała się optymalizacja Bayesa (BayesSearchCV), która jest alternatywą dla Grid Search przy szukaniu optymalnych parametrów w XGBoost.

Przy pisaniu kodu korzystałam z czatu GPT, jednak pomagał on w pisaniu małych fragmentów kodu. W moim doświadczeniu czat najlepiej radzi sobie z małymi zadaniami, które następnie trzeba włączyć do własnego programu.


# 2. Metodyka

1. Program składa się z 3 głównych części: main, preprocess_data i train_model_bayes_search (oraz nieużyty finalnie train_model, który używał GridSearcha. Poniżej opisałam co zawiera każda część.


2. Przetwarzanie danych:

- Dodanie kolumny roku na podstawie 'date_active'.

- Poprawienie typów danych na prawidłowe (object na bool), zamiana brakujących i nieprawidłowych wartości (np liczba pokoi <1, liczba metrów <0) na np.nan.

- W kolumnach o typie danych "bool" zaimputowanie "FALSE" dla wszystkich brakujących wartości.


- Dla flat_rooms
  - imputacja wartości "1"  dla rekordów z "kawale*" w opisie.

  - Imputacja  flat_rooms na podstawie ad_title (rekordy z fragmentem "pokoj*") przy pomocy biblioteki "re" funkcji extract_room_count.

  - Imputacja flat_rooms dla rekordów z flat_area > 0 na podstawie knn = 3 z flat_area.

  - Imputacja średnią dla pozostałych braków w flat_rooms.

- Dla flat_area:
  - Imputacja metodą knn = 3 na podstawie flat_rooms. Założenie, że mieszkania o tej samej liczbie pokoi mogą mieć podobną wielkość.

- Dla quarter:
   - Imputacja brakującej kolumny quarter na podstawie ad_title. Jeżeli ad_title zawiera słowo należace do zbioru dzielnic z kolumny quarter, imputujemy je.

   - Dla pozostałych dzielnic, imputacja dzielnicy "inne"


3. Trenowanie modelu:

- Najpierw użyłam funkcji liniowej.
- Następnie Decision Tree i Random Forest przy pomocy Grid Search z pliku train_model.
- Następnie użyłam Random Search, który jest szybszy w szukaniu najlepszych parametrów od Grid Search
- Ostatecznie wybrałam BayesSearchCV (funkcja znajduje się w pliku train_model_bayes_search), który okazał się najlepszy w szukaniu optymalnych parametrów, jednak wyszukanie ich zabiera dużo czasu.





# 3. Wyniki

Trenowanie modelu i 10 predykcji. Wyniki zależały od sposobu imputacji danych i wybranego modelu. W nawiasach podałam wyniki MSE dla pliku train.

1. Funkcja Liniowa (182225.62) dla
        'flat_area', 'flat_rooms', 'year_activ'

Imputacja flat_rooms z knn flat_area w przypadku rekordów z flat_area>0. Dla reszty średnia.
Flat_area imputowane przy pomocy knn z flat_rooms.

2. Grid Search dla
        'flat_area', 'flat_rooms', 'year_activ'
- Decision Tree (164590.03)
- Random Forest (156813.83) dla parametrów: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}

3. Imputacja flat_rooms z ad_title (kawale* i regex dla większej liczby pokoi).
Dla reszty rekordów flat_rooms ze średniej.
Imputacja flat_area z knn flat_rooms.

4. Dodanie kolumny 'year_active' z 'date_active'. Uwzględnienie jej w budowaniu modelu.

4. Random Search - najlepszy wynik (109019.76) dla Random Forest dla parametrów.
Dodanie
        'flat_rent', 'flat_deposit', 'flat_for_students', 'building_floor_num', 'flat_balcony', 'flat_utility_room', 'flat_garage',
        'flat_basement', 'flat_garden', 'flat_tarrace', 'flat_lift', 'flat_two_level', 'flat_kitchen_sep',
        'flat_air_cond', 'flat_nonsmokers', 'flat_washmachine', 'flat_dishwasher', 'flat_fridge',
        'flat_cooker', 'flat_oven', 'flat_internet', 'flat_television', 'flat_anti_blinds',
        'flat_monitoring', 'flat_closed_area'

{'max_depth': 22, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

5. Imputacja brakujących 'quarter' na podstawie ad_title. Dla reszty rekordów imputacja 'other'. Użycie danych typu dummies.

6. BayesSearchCV (XGBoost) w train_model_bayes_search:
- Decision Tree: (130202.42)
- Random Forest: (86920.66) dla parametrów: {'max_depth': 45, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

# 4. Podsumowanie

Bardzo częstym problemem w analizie danych są braki danych. W tym projekcie wykorzystałam wiedzę z zakresu analizy danych niekompletnych w celu imputacji brakujących danych różnymi metodami.

Przed analizą i imputacją danych konieczne jest bardzo dobre zrozumienie danych i ich kontekstu. Bez tego nie jesteśmy również w stanie efektywnie korzystać z LLMów, które potrzebują jasnych instrukcji i fragmentowania kodu.
Dobre zrozumienie danych, wybranie metod do imputacji, metody budowy modelu i umiejętność korzystania z LLM-ów, umożliwiły mi poprawę modelu i otrzymanie wyniku 112342.75 na Kaggle.

# 5. Kody do odtworzenia wyników

In [ ]:
!pip install scikit-learn
!pip install scikit-optimize # instalacja wymaganych modułów


import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
import re
from sklearn.tree import DecisionTreeRegressor
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from skopt import BayesSearchCV

#  Funkcja przetwarzania danych
def preprocess_data(data, columns_object_dtype, columns_with_conditions):
    """Przetwarza dane, wypełnia braki i przygotowuje je do modelowania."""
    data['year_activ'] = pd.to_datetime(data['date_activ'], errors = 'coerce').dt.year
    data['year_activ'] = data['year_activ'].fillna(data['year_activ'].mode()[0])


    # Zamiana wartości w kolumnie na typ bool
    for column in data.select_dtypes(include='object').columns:
        if column not in columns_object_dtype:
            pd.set_option('future.no_silent_downcasting', True)
            data[column] = data[column].fillna(False).astype(bool)

    # Zamiana braków danych lub wartości mniejszych od 0
    data = data.replace([None, '', 'NA', 'N/A', 'null'], np.nan)
    for column, condition in columns_with_conditions.items():
        data.loc[condition(data[column]), column] = np.nan

    # Sprawdzenie ad_title i imputacja wartości "1" w kolumnie "flat_rooms" dla kawalerek
    data.loc[data['flat_rooms'].isna() & data['ad_title'].str.contains('(?i)kawale', na=False), 'flat_rooms'] = 1

    # Sprawdzanie ad_title i imputacja liczby pokoi na podstawie liczby przed "pok"
    def extract_room_count(title):
        match = re.search(r'(\d)\s*-?\s*pok(?:ojowe|oje|\.|\s)', title, re.IGNORECASE)
        return int(match.group(1)) if match else np.nan

    data.loc[data['flat_rooms'].isna(), 'flat_rooms'] = data['ad_title'].apply(
        lambda x: extract_room_count(x) if isinstance(x, str) else np.nan)

    # Imputacja flat_rooms na podstawie knn z flat_area
    mask_missing_rooms = data["flat_rooms"].isna() & (data["flat_area"] > 0)
    data_knn_rooms = data[['flat_area', 'flat_rooms']]
    imputer = KNNImputer(n_neighbors=3)
    data_knn_imputed_rooms = imputer.fit_transform(data_knn_rooms)
    data.loc[mask_missing_rooms, "flat_rooms"] = data_knn_imputed_rooms[mask_missing_rooms, 1]

    # Imputacja średnią dla pozostałych braków w flat_rooms
    mean_imputer = SimpleImputer(strategy='mean')
    data['flat_rooms'] = mean_imputer.fit_transform(data[['flat_rooms']])

    # Imputacja flat_area na podstawie flat_rooms za pomocą KNNImputer
    mask_missing_area = data["flat_area"].isna() & (data["flat_rooms"] > 0)
    data_knn_area = data[['flat_rooms', 'flat_area']]
    imputer_area = KNNImputer(n_neighbors=3)
    data_knn_imputed_area = imputer_area.fit_transform(data_knn_area)
    data.loc[mask_missing_area, "flat_area"] = data_knn_imputed_area[mask_missing_area, 1]


    #Imutacja brakujacych wartości w kolumnie 'quarter'
    unique_quarters = data['quarter'].dropna().unique()
    def extract_quarter_from_title(ad_title):
        if isinstance(ad_title, str):
            for quarter in unique_quarters:
                if str(quarter).lower() in ad_title.lower():
                    return quarter
        return np.nan
    data['quarter'] = data['quarter'].fillna(data['ad_title'].apply(extract_quarter_from_title))
    # Imputacja brakujących wartości w kolumnie 'quarter'
    data['quarter'] = data['quarter'].fillna('inne')

    return data


#FUnkcja trenowania modelu
def train_and_evaluate_models_with_bayessearch(models_with_params, X_train, y_train, X_test, y_test):
    """Trenuje modele z BayesSearchCV i ocenia je za pomocą MSE."""
    results = {}
    predictions_dict = {}
    for name, (model, search_space) in models_with_params.items():
            bayes_search = BayesSearchCV(
                estimator=model,
                search_spaces=search_space,
                scoring='neg_mean_squared_error',
                n_iter=20,  # Liczba iteracji optymalizacji
                cv=3,
                verbose=1,
                random_state=42
            )
            bayes_search.fit(X_train, y_train)
            best_model = bayes_search.best_estimator_
            predictions = best_model.predict(X_test)
            mse = mean_squared_error(y_test, predictions)
            results[name] = mse
            predictions_dict[name] = predictions
            print(f"{name} - Najlepsze parametry: {bayes_search.best_params_}")
            print(f"Najlepsze MSE: {-bayes_search.best_score_}")
    return results, predictions_dict



if __name__ == "__main__":



    #wczytanie danych do użycia funkcji preprocess_data
    train_file_path= 'pzn-rent-train.csv'
    test_final_file_path = 'pzn-rent-test.csv'
    train_data = pd.read_csv(train_file_path)

    # Przygotowanie danych do funkcji preprocess_data
    columns_object_dtype = ['ad_title', 'date_activ', 'date_modif', 'date_expire', 'quarter']
    columns_with_conditions = {
        'flat_area': lambda x: x <= 0,
        'flat_rooms': lambda x: x <= 0,
        'flat_rent': lambda x: x < 0,
        'flat_deposit': lambda x: x < 0
    }

    #przetworzenie danych (po else) lub pobranie wcześniej przetowrzonego pliku z danymi
    modified_train_file = 'pzn-rent-train-modified.csv'
    if os.path.exists(modified_train_file):
        train_data = pd.read_csv("pzn-rent-train-modified.csv")
        print("wczytano przetwrzone dane z pliku")
    else:
        train_data = preprocess_data(train_data, columns_object_dtype, columns_with_conditions)
        train_data.to_csv(modified_train_file, index=False)
        print("Przetworzono dane i zapisano do pliku.")

    #zamiana danych w kolumnie "quarter" na typ dummies
    train_data = pd.get_dummies(train_data, columns=['quarter'], drop_first=True)

    #przygotowanie cech
    additional_features = [
        'flat_rent', 'flat_deposit', 'flat_for_students', 'building_floor_num', 'flat_balcony', 'flat_utility_room', 'flat_garage',
        'flat_basement', 'flat_garden', 'flat_tarrace', 'flat_lift', 'flat_two_level', 'flat_kitchen_sep',
        'flat_air_cond', 'flat_nonsmokers', 'flat_washmachine', 'flat_dishwasher', 'flat_fridge',
        'flat_cooker', 'flat_oven', 'flat_internet', 'flat_television', 'flat_anti_blinds',
        'flat_monitoring', 'flat_closed_area'
    ]
    features = ['flat_area', 'flat_rooms', 'year_activ'] + [col for col in train_data.columns if col.startswith('quarter_')] + additional_features
    print("Wykorzystywane cechy:", features + additional_features)
    target = 'price'
    train_data = train_data.dropna(subset=features + [target])
    X = train_data[features]
    y = train_data[target]

    # podział na zbiór treningowy i testowy pliky train_data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Definicja modeli
    models_with_params = {
         'Decision Tree': (DecisionTreeRegressor(random_state=42), {
             'max_depth': (10, 50),
             'min_samples_split': (2, 10),
             'min_samples_leaf': (1, 5)
        }),
        'Random Forest': (RandomForestRegressor(random_state=42), {
            'n_estimators': (50, 300),
            'max_depth': (10, 50),
            'min_samples_split': (2, 10),
            'min_samples_leaf': (1, 5)
        })
    }


    # Trenowanie modeli
    results, predictions = train_and_evaluate_models_with_bayessearch(models_with_params, X_train, y_train, X_test, y_test)

    # Porównanie wyników
    print("Porównanie MSE dla modeli:")
    for model_name, mse in results.items():
        print(f"{model_name}: {mse:.2f}")

    # Wybór najlepszego modelu
    best_model_name = min(results, key=results.get)
    print(f"Najlepszy model: {best_model_name} z MSE: {results[best_model_name]:.2f}")
    # Analiza ważności cech dla najlepszego modelu (np. Random Forest)
    best_model = models_with_params[best_model_name][0]


    # Wczytanie danych testowych i predykcja
    test_final_data = pd.read_csv(test_final_file_path)
    test_final_data = preprocess_data(test_final_data, columns_object_dtype, columns_with_conditions)
    test_final_data = pd.get_dummies(test_final_data, columns=['quarter'], drop_first=True)
    test_final_data = test_final_data.reindex(columns=X_train.columns, fill_value=0)

    final_model = models_with_params[best_model_name][0]
    final_model.fit(X_test, y_test)
    predicted_prices = final_model.predict(test_final_data)

    # Zapis wyników
    output_test_final_data = pd.DataFrame({
        'ID': range(1, len(predicted_prices) + 1),
        'TARGET': predicted_prices
    })
    output_test_final_data.to_csv('pzn-rent-test-predicted.csv', index=False)
    print("Predykcja zakończona. Wyniki zapisano w 'pzn-rent-test-predicted.csv'")
